In [1]:
import pandas as pd
import os
os.chdir('Old_speaker_labels/')


In [2]:
label_dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]
label_dir_names

['Bill_Clinton_corrected_SMcL',
 'Gloria_Steinem_corrected_Ryan',
 'Hillary_Clinton_corrected_Ryan',
 'James_Baldwin_corrected_Ryan',
 'Julia_Child_corrected_Ryan',
 'LBJ_corrected_Ryan',
 'Malcolm_X_corrected_Ryan',
 'Malcolm_X_corrected_SMcL',
 'Ossie_Davis_Ryan',
 'Richard_Nixon_corrected_Ryan',
 'Ronald_Reagan_corrected_Ryan']

In [3]:
for dir_name in label_dir_names:
    
    main_label = dir_name.split('_corrected_')[0].replace('_',' ')
    
    label_filenames = os.listdir(dir_name)
    label_dataframes = []

    for filename in [item for item in label_filenames if item!='.DS_Store']:
        #filename = label_filenames[5]
        basename = filename.replace('_corrected.csv','')
        
        label_rows = open(os.path.join(dir_name,filename)).read().splitlines()

        label_rows = [item for item in label_rows if (item!='')&(item[:2]!='##')]
        
        label_rows = [item.split(',') for item in label_rows]

        if len(label_rows)>0:
            label_rows_filtered = []
            
            for row in label_rows:
                
                #if len(row)<3:       #debugging
                #    print(dir_name)
                #    print(filename)

                if float(row[2])!= 0.0:  ## filtering out accidental empty labels
                    if len(row)==3:
                        row.append(main_label)
                    if len(row)!=4:
                        print("ERROR: wrong number of cells")
                        print(dir_name)
                        print(filename)
                        print(row)
                    else:
                        label_rows_filtered.append(row)
                        
            label_df = pd.DataFrame(label_rows_filtered, columns = ["Start time","Num_ID_discard","Duration","Label"])
            label_df["Media file basename"] = basename
            if '_Ryan' in dir_name:
                label_df["Labeled by"] = "Ryan Blake"
            elif '_SMcL' in dir_name:
                label_df["Labeled by"] = "Stephen R. McLaughlin"
            label_dataframes.append(label_df)

    if len(label_dataframes)>0:
        out_table = pd.concat(label_dataframes) # concatenating tables
        out_table = out_table[["Media file basename","Start time","Duration","Label","Labeled by"]] #rearranging columns
        out_table.to_csv(dir_name.replace('/','')+'.csv', index=False)